# GPU/CPU and path directory settings

In [13]:
import DeviceDir

DIR, RESULTS_DIR = DeviceDir.get_directory()
device, NUM_PROCESSORS = DeviceDir.get_device()

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A10
cuda
Cpu count:  32


# Library

In [3]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
dataset = dgl.data.CoraGraphDataset(DIR)
print("Number of categories:", dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [5]:
g = dataset[0]
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [6]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


In [7]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [8]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.184 (best 0.184), test acc: 0.187 (best 0.187)
In epoch 5, loss: 1.890, val acc: 0.614 (best 0.614), test acc: 0.583 (best 0.583)
In epoch 10, loss: 1.809, val acc: 0.734 (best 0.734), test acc: 0.736 (best 0.736)
In epoch 15, loss: 1.704, val acc: 0.730 (best 0.748), test acc: 0.714 (best 0.744)
In epoch 20, loss: 1.575, val acc: 0.744 (best 0.748), test acc: 0.739 (best 0.744)
In epoch 25, loss: 1.424, val acc: 0.748 (best 0.750), test acc: 0.752 (best 0.747)
In epoch 30, loss: 1.257, val acc: 0.748 (best 0.752), test acc: 0.744 (best 0.750)
In epoch 35, loss: 1.082, val acc: 0.756 (best 0.756), test acc: 0.751 (best 0.751)
In epoch 40, loss: 0.910, val acc: 0.764 (best 0.764), test acc: 0.756 (best 0.754)
In epoch 45, loss: 0.749, val acc: 0.764 (best 0.764), test acc: 0.756 (best 0.754)
In epoch 50, loss: 0.607, val acc: 0.766 (best 0.766), test acc: 0.757 (best 0.755)
In epoch 55, loss: 0.488, val acc: 0.764 (best 0.766), test acc: 0.765 (best 0

In [12]:
g = g.to(device)
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to(device)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.136 (best 0.136), test acc: 0.120 (best 0.120)
In epoch 5, loss: 1.890, val acc: 0.370 (best 0.370), test acc: 0.366 (best 0.366)
In epoch 10, loss: 1.807, val acc: 0.486 (best 0.486), test acc: 0.467 (best 0.467)
In epoch 15, loss: 1.702, val acc: 0.520 (best 0.520), test acc: 0.518 (best 0.518)
In epoch 20, loss: 1.577, val acc: 0.598 (best 0.598), test acc: 0.592 (best 0.592)
In epoch 25, loss: 1.433, val acc: 0.600 (best 0.600), test acc: 0.616 (best 0.616)
In epoch 30, loss: 1.277, val acc: 0.636 (best 0.636), test acc: 0.647 (best 0.647)
In epoch 35, loss: 1.113, val acc: 0.668 (best 0.668), test acc: 0.690 (best 0.690)
In epoch 40, loss: 0.952, val acc: 0.690 (best 0.690), test acc: 0.720 (best 0.720)
In epoch 45, loss: 0.799, val acc: 0.714 (best 0.714), test acc: 0.738 (best 0.738)
In epoch 50, loss: 0.660, val acc: 0.736 (best 0.736), test acc: 0.748 (best 0.748)
In epoch 55, loss: 0.540, val acc: 0.748 (best 0.748), test acc: 0.758 (best 0

In [10]:
g1 = dgl.graph(([0, 1, 1, 2, 3], [1, 2, 3, 0, 0]))

In [11]:
g1.edata['p'] = torch.FloatTensor([1, 0, 1, 1, 1])     # disallow going from 1 to 2
dgl.sampling.random_walk(g1, [0, 1, 2, 0], length=2, prob='p')

(tensor([[0, 1, 3],
         [1, 3, 0],
         [2, 0, 1],
         [0, 1, 3]]),
 tensor([0, 0, 0]))